<a href="https://colab.research.google.com/github/ansul1214/Open_Project_Winter_2025/blob/main/W1_Quantum_Tomography_23113029.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Environment Setup

In [1]:
!python -m pip install  pennylane numpy scipy pandas plotly tqdm nbformat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 66.9 MB/s eta 0:00:00


In [2]:
!python -m pip install jax~=0.6.0 jaxlib~=0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.7.2
    Uninstalling jaxlib-0.7.2:
      Successfully uninstalled jaxlib-0.7.2
  Attempting uninstall: jax
    Found existing installation: jax 0.7.2
    Uninstalling jax-0.7.2:
      Successfully uninstalled jax-0.7.2


In [3]:
import pennylane as qml
import numpy as np
import json
from typing import Dict, Any

# Task 2: Measurement Theory Pointer

To-Do: Prepare at minimum the computational basis (|0⟩, |1⟩), the Hadamard basis (|+⟩, |−⟩), and one phase-offset state (e.g., ( |0⟩ + i |1⟩ ) / √2). Document how you synthesize each state in circuit form and store a textual or JSON summary of the gates used. You may optionally include mixed states by applying depolarizing or amplitude damping channels.

|0> : [1 0].T - 0 bit Dirac notation


|1> : [0 1].T - 1 bit Dirac notation

Hadmard Basis:

These are the linear combinations of the standard basis |0> and |1>.

|+> = 1/√2(|0>+|1>)

|-> = 1/√2(|0>-|1>)

The |1> can be represented as a circuit of the not operator on the zero bit.

|1>=X|0>

X=array([[1, 0],
       [0, 1]])

For |+> = H|0>

 |-> = HS|0>

 For |0+i1>/√2
   
   |0+i1>/√2 = HS|0>


In [4]:
dev = qml.device("default.qubit", wires=1)

In [21]:
def prepare_state(state):
    if state == "|0>":
        pass
    elif state == "|1>":
        qml.PauliX(0)
    elif state == "|+>":
        qml.Hadamard(0)
    elif state == "|->":
        qml.PauliX(0)
        qml.Hadamard(0)
    elif state == "|0+i1>/√2":
        qml.Hadamard(0)
        qml.S(0)
    else:
        raise ValueError("Invalid state")

In [6]:
state_preparation_summary = {
    "|0>": [],
    "|1>": ["PauliX"],
    "|+>": ["Hadamard"],
    "|->": ["PauliX", "Hadamard"],
    "|0+i1>/√2": ["Hadamard", "S"]
}

with open("state_preparation.json", "w") as f:
    json.dump(state_preparation_summary, f, indent=2)


In [7]:
@qml.qnode(dev)
def noisy_plus_state(p=0.1):
    qml.Hadamard(0)
    qml.DepolarizingChannel(p, wires=0)
    return qml.density_matrix(wires=0)

In [9]:
import pathlib

def build_measurement_model(config_path: pathlib.Path) -> Dict[str, Any]:
    """
    Stub for constructing or loading the measurement operators you plan to use.
    Populate the return value with operator definitions, normalization checks, and metadata.
    """
    X=np.array([[0,1],[1,0]],dtype=complex)
    Y=np.array([[0,-1j],[1j,0]],dtype=complex)
    Z=np.array([[1,0],[0,-1]],dtype=complex)
    I=np.array([[1,0],[0,1]],dtype=complex)
    P0=np.array([[1,0],[0,0]],dtype=complex)
    P1=np.array([[0,0],[0,1]],dtype=complex)
    completeness=np.allclose(P0+P1,I)
    return {
        "type": "pauli_projective",
        "operators":{"X":X,"Y":Y,"Z":Z,"I":I},
        "Projections":{"P0":P0,"P1":P1},
        "completeness":completeness,
        "metadata": {
            "framework": "PennyLane",
            "description": "Single-qubit Pauli projective measurement model"
        }
    }


    # TODO: implement SIC POVM or Pauli projective operator assembly here.
    raise NotImplementedError("Create your measurement operator assembly here.")

# Task 3: QST Data Generation

In [43]:
STATE_ID_MAP = {
    "|0>": "zero",
    "|1>": "one",
    "|+>": "plus",
    "|->": "minus",
    "|0+i1>/√2": "phase"
}

In [44]:
from dataclasses import dataclass
from typing import List
import pathlib

In [45]:
@dataclass
class DatasetVariant:
    name: str
    circuit_summary: str
    measurement_model: str
    measurement_data_path: pathlib.Path
    metadata_path: pathlib.Path
    density_matrix_path: pathlib.Path

In [46]:
@dataclass
class DatasetVariant:
    name: str
    circuit_summary: str
    measurement_model: str
    measurement_data_path: pathlib.Path
    metadata_path: pathlib.Path
    density_matrix_path: pathlib.Path

In [47]:
SHOTS = 2000
NUM_QUBITS = 1   # change to >1 later

dev_counts = qml.device(
    "default.qubit",
    wires=NUM_QUBITS,
)

dev_dm = qml.device(
    "default.qubit",
    wires=NUM_QUBITS
)


In [48]:
@qml.set_shots(SHOTS)
@qml.qnode(dev_counts)
def measure_pauli_string(state_label: str, pauli_string: str):
    prepare_state(state_label)

    for wire, p in enumerate(pauli_string):
        if p == "X":
            qml.Hadamard(wire)
        elif p == "Y":
            qml.PhaseShift(np.pi / 2, wire)
            qml.Hadamard(wire)
        elif p == "Z":
            pass
        elif p == "I":
            pass

    return qml.counts(wires=range(NUM_QUBITS))


In [49]:
@qml.qnode(dev_dm)
def get_true_density_matrix(state_label: str):
    prepare_state(state_label)
    return qml.density_matrix(wires=range(NUM_QUBITS))

In [50]:
@qml.qnode(dev_dm)
def get_true_density_matrix(state_label: str):
    prepare_state(state_label)
    return qml.density_matrix(wires=range(NUM_QUBITS))

In [51]:
def normalize_counts(counts: dict):
    total = sum(counts.values())
    return {k: v / total for k, v in counts.items()}


In [57]:
def generate_measurement_dataset(variants: List[DatasetVariant]) -> None:
    """
    Populate each variant with measurement outcomes, metadata,
    and ground-truth density matrices.
    Designed to scale from single-qubit to multi-qubit systems.
    """

    # Measurement settings (extendable)
    pauli_settings = ["X", "Y", "Z"] if NUM_QUBITS == 1 else [
        "XX", "YY", "ZZ", "XY", "XZ", "YZ"
    ]


    for variant in variants:
        print("Processing state:",variant.name)
        print("Saving to:",variant.measurement_data_path)
        state = variant.name
        measurement_results = {}

        for setting in pauli_settings:
            counts = measure_pauli_string(state, setting)
            measurement_results[setting] = {
                "counts": counts,
                "probabilities": normalize_counts(counts)
            }

        # Ensure directories exist
        variant.measurement_data_path.parent.mkdir(parents=True, exist_ok=True)

        # Save measurement data
        np.save(
            variant.measurement_data_path,
            measurement_results,
            allow_pickle=True
        )

        # Save ground truth density matrix
        rho_true = get_true_density_matrix(state)
        np.save(variant.density_matrix_path, rho_true)

        # Save metadata
        metadata = {
            "state": state,
            "num_qubits": NUM_QUBITS,
            "measurement_model": variant.measurement_model,
            "pauli_settings": pauli_settings,
            "shots": SHOTS
        }

        with open(variant.metadata_path, "w") as f:
            json.dump(metadata, f, indent=2)


In [58]:
base_dir = pathlib.Path("data")

state_id = STATE_ID_MAP[state]

variants = [
    DatasetVariant(
        name=s,
        circuit_summary=f"State preparation for {s}",
        measurement_model="Pauli projective",
        measurement_data_path=base_dir / f"single_qubit_{STATE_ID_MAP[s]}_measurements.npy",
        metadata_path=base_dir / f"single_qubit_{STATE_ID_MAP[s]}_metadata.json",
        density_matrix_path=base_dir / f"single_qubit_{STATE_ID_MAP[s]}_rho_true.npy"
    )
    for s in states
]
generate_measurement_dataset(variants)

Processing state: |0>
Saving to: data/single_qubit_zero_measurements.npy
Processing state: |1>
Saving to: data/single_qubit_one_measurements.npy
Processing state: |+>
Saving to: data/single_qubit_plus_measurements.npy
Processing state: |->
Saving to: data/single_qubit_minus_measurements.npy
Processing state: |0+i1>/√2
Saving to: data/single_qubit_phase_measurements.npy


In [59]:
print("Number of variants:", len(variants))
print(variants)

Number of variants: 5
[DatasetVariant(name='|0>', circuit_summary='State preparation for |0>', measurement_model='Pauli projective', measurement_data_path=PosixPath('data/single_qubit_zero_measurements.npy'), metadata_path=PosixPath('data/single_qubit_zero_metadata.json'), density_matrix_path=PosixPath('data/single_qubit_zero_rho_true.npy')), DatasetVariant(name='|1>', circuit_summary='State preparation for |1>', measurement_model='Pauli projective', measurement_data_path=PosixPath('data/single_qubit_one_measurements.npy'), metadata_path=PosixPath('data/single_qubit_one_metadata.json'), density_matrix_path=PosixPath('data/single_qubit_one_rho_true.npy')), DatasetVariant(name='|+>', circuit_summary='State preparation for |+>', measurement_model='Pauli projective', measurement_data_path=PosixPath('data/single_qubit_plus_measurements.npy'), metadata_path=PosixPath('data/single_qubit_plus_metadata.json'), density_matrix_path=PosixPath('data/single_qubit_plus_rho_true.npy')), DatasetVariant

In [61]:
!ls data
!ls data/single_qubit

 single_qubit			       single_qubit_one_measurements.npy
'single_qubit_|0+i1>'		       single_qubit_one_metadata.json
'single_qubit_|0>_measurements.npy'    single_qubit_one_rho_true.npy
'single_qubit_|0>_metadata.json'       single_qubit_phase_measurements.npy
'single_qubit_|0>_rho_true.npy'        single_qubit_phase_metadata.json
'single_qubit_|1>_measurements.npy'    single_qubit_phase_rho_true.npy
'single_qubit_|1>_metadata.json'       single_qubit_plus_measurements.npy
'single_qubit_|1>_rho_true.npy'        single_qubit_plus_metadata.json
'single_qubit_|+>_measurements.npy'    single_qubit_plus_rho_true.npy
'single_qubit_|->_measurements.npy'   'single_qubit_|+>_rho_true.npy'
'single_qubit_|+>_metadata.json'      'single_qubit_|->_rho_true.npy'
'single_qubit_|->_metadata.json'       single_qubit_zero_measurements.npy
 single_qubit_minus_measurements.npy   single_qubit_zero_metadata.json
 single_qubit_minus_metadata.json      single_qubit_zero_rho_true.npy
 single_qubit_minus_rho_t

# Task 4: Single Qubit Tomography

In [74]:
def state_to_id(state_label: str) -> str:
    return {
        "|0>": "zero",
        "|1>": "one",
        "|+>": "plus",
        "|->": "minus",
        "|0+i1>/√2": "phase"
    }[state_label]

In [25]:
import pathlib
import numpy as np
import json

In [68]:
DATA_DIR = pathlib.Path("data")

states = ["|0>", "|1>", "|+>", "|->", "|0+i1>/√2"]


In [70]:
def load_measurement_data(state_label):
    path = DATA_DIR / f"single_qubit_{state_label}_measurements.npy"
    print("Loading:", path)
    return np.load(path, allow_pickle=True).item()


In [71]:
def load_true_density_matrix(state_label):
    path = DATA_DIR / f"single_qubit_{state_label}_rho_true.npy"
    print("Loading:", path)
    return np.load(path)


In [75]:
def linear_inversion_tomography(measurement_data):
    expX = measurement_data["X"]["probabilities"].get("0", 0) \
         - measurement_data["X"]["probabilities"].get("1", 0)

    expY = measurement_data["Y"]["probabilities"].get("0", 0) \
         - measurement_data["Y"]["probabilities"].get("1", 0)

    expZ = measurement_data["Z"]["probabilities"].get("0", 0) \
         - measurement_data["Z"]["probabilities"].get("1", 0)

    rho = 0.5 * (I + expX * X + expY * Y + expZ * Z)
    return rho

In [76]:
RECON_DIR = DATA_DIR / "reconstructions"
RECON_DIR.mkdir(parents=True, exist_ok=True)

reconstruction_results = {}

In [77]:
X=np.array([[0,1],[1,0]],dtype=complex)
Y=np.array([[0,-1j],[1j,0]],dtype=complex)
Z=np.array([[1,0],[0,-1]],dtype=complex)
I=np.array([[1,0],[0,1]],dtype=complex)

for state in states:
    meas = load_measurement_data(state)
    rho_recon = linear_inversion_tomography(meas)
    rho_true = load_true_density_matrix(state)

    state_id = state_to_id(state)
    np.save(RECON_DIR / f"{state_id}_rho_reconstructed.npy", rho_recon)

    reconstruction_results[state_id] = {
        "rho_reconstructed": rho_recon,
        "rho_true": rho_true
    }

Loading: data/single_qubit_|0>_measurements.npy
Loading: data/single_qubit_|0>_rho_true.npy
Loading: data/single_qubit_|1>_measurements.npy
Loading: data/single_qubit_|1>_rho_true.npy
Loading: data/single_qubit_|+>_measurements.npy
Loading: data/single_qubit_|+>_rho_true.npy
Loading: data/single_qubit_|->_measurements.npy
Loading: data/single_qubit_|->_rho_true.npy
Loading: data/single_qubit_|0+i1>/√2_measurements.npy
Loading: data/single_qubit_|0+i1>/√2_rho_true.npy


In [ ]:
from scipy.linalg import sqrtm